In [37]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error, r2_score 
import numpy as np


In [38]:
house = pd.read_csv("kc_house_data.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

# Scale the data so that each feature has mean 0 and standard deviation 1.

# Divide price by 1000 
house['price'] = house['price'] / 1000
train['price'] = train['price'] / 1000
test['price'] = test['price'] / 1000

In [43]:
# removing not needed columns 

cols_to_drop = ['price', 'zipcode', 'id', 'date', 'Unamed: 0']
# Also drop 'Unnamed: 0' if present
for df in [train, test]:
    if 'Unnamed: 0' in df.columns:
        cols_to_drop_df = cols_to_drop + ['Unnamed: 0']
    else:
        cols_to_drop_df = cols_to_drop

feature_cols = [c for c in train.columns if c not in cols_to_drop + ['Unnamed: 0']]

# Standardize feature columns
scaler = StandardScaler()
train[feature_cols] = scaler.fit_transform(train[feature_cols])
test[feature_cols] = scaler.transform(test[feature_cols])
house[feature_cols] = scaler.transform(house[feature_cols])

In [44]:
# checking to see if the mean is 0 and std is 1
print("Training set feature means:")
print(train[feature_cols].mean().round(6))


print("Training set feature stds:")
print(train[feature_cols].std().round(6))

Training set feature means:
bedrooms        -0.0
bathrooms        0.0
sqft_living      0.0
sqft_lot        -0.0
floors          -0.0
waterfront      -0.0
view            -0.0
condition       -0.0
grade            0.0
sqft_above      -0.0
sqft_basement    0.0
yr_built         0.0
yr_renovated     0.0
lat             -0.0
long            -0.0
sqft_living15   -0.0
sqft_lot15       0.0
dtype: float64
Training set feature stds:
bedrooms         1.0005
bathrooms        1.0005
sqft_living      1.0005
sqft_lot         1.0005
floors           1.0005
waterfront       1.0005
view             1.0005
condition        1.0005
grade            1.0005
sqft_above       1.0005
sqft_basement    1.0005
yr_built         1.0005
yr_renovated     1.0005
lat              1.0005
long             1.0005
sqft_living15    1.0005
sqft_lot15       1.0005
dtype: float64


In [46]:
house.dtypes

id                 int64
date              object
price            float64
bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot         float64
floors           float64
waterfront       float64
view             float64
condition        float64
grade            float64
sqft_above       float64
sqft_basement    float64
yr_built         float64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15    float64
sqft_lot15       float64
dtype: object

# *[C]* Problem 2:  Linear regression (15 points)

In this problem, you will use an existing package of your choice for training and testing a linear regression model for the house prediction
dataset.

1. Use an existing package to train a multiple linear regression model on the training set using all the features (except the ones excluded
above). Report the coefficients of the linear regression models and the following metrics on the training data: (1) MSE metric; (2)
$R^2$ metric.


In [47]:
# X is what you use to predict
X_train = train[feature_cols]

# y is the TARGET (what you want to predict)
y_train = train['price'].values

In [48]:
# Using LinearRegression from SKLEARN I will solve this problem 
model = LinearRegression()

# Fit the model 
model.fit(X_train, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [7]:
# Imporantance of the features 
cof = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_
})

print("Intercept:", round(model.intercept_, 4))
print()
cof.sort_values(by='Coefficient', ascending=False, key=abs)

Intercept: 520.4148



,Feature,Coefficient
8,grade,92.231475
13,lat,78.375737
11,yr_built,-67.643117
5,waterfront,63.742900
2,sqft_living,56.748837
9,sqft_above,48.290089
6,view,48.200109
15,sqft_living15,45.577658
10,sqft_basement,27.137032
1,bathrooms,18.527633


In [8]:
# Predict house prices for the training set
y_pred_train = model.predict(X_train)

In [9]:
# MSE and R squared Metrics
mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train,y_pred_train)
print('mse:', round(mse,2), 'r2:', round(r2,2) )

mse: 31486.17 r2: 0.73


2. Evaluate the model on the testing set. Report the MSE and $R^2$ metrics on the testing set.


In [10]:
# X is what you use to predict
X_test = test[feature_cols]

# y is the TARGET (what you want to predict)
y_test = test['price'].values

# Use the ALREADY TRAINED model to predict on the test set
y_pred_test = model.predict(X_test)

In [11]:
# RSE and MSE metrics
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
print('mse:', round(mse_test,2), 'r2:', round(r2_test,2))

mse: 57628.15 r2: 0.65


3. Interpret the results in your own words. Which features contribute mostly to the linear regression model? Is the model fitting the data
well? How large is the model error? How do the training and testing MSE relate?

Since the features are standardized, the coefficents are directly comparable in magnitude a larger absolute coefficient means the feature has a greater impact on the predicted price. The features that contribute most to the model are the ones with the largest absolute coefficients (sqft_living, grade, lat, waterfront, view).

The model fits the data reasonably well with an R^2 of 0.73 on training. This means about 73% of the variance in house prices can be explained. Since we divided price by 1000, the MSE values are much smaller and more interpertable. The testing MSE is higher since the model was optimized on the training data and may not generalize perfectly to data that hasn't been seen. The R^2 is also slightly lower on the test set, highlighting a small drop in performance on new data. 

# *[C]* Problem 3:  Implementing closed-form solution for linear regression (15 points)

In this problem, you will implement your own linear regression model, using the closed-form solution we derived in class. You will also
compare your model with the one trained with the package in Problem 2 on the same house price prediction dataset.

- Implement the closed-from solution for multiple linear regression using matrix operations and train a model on the training set. Write
a function to predict the response on a new testing point.

In [12]:
class MyLinearRegression:
    
    def fit(self, X, y):
        # Add intercept column
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        
        # Closed-form solution
        self.beta = np.linalg.solve(X_b.T @ X_b, X_b.T @ y)
    
    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b @ self.beta


In [13]:
# Redefine features for Problem 3 
X_train_all = train[feature_cols].values
y_train_all = train['price'].values

X_test_all = test[feature_cols].values
y_test_all = test['price'].values

# Train closed-form model on training data
my_model = MyLinearRegression()
my_model.fit(X_train_all, y_train_all)

In [14]:
# Report coefficients
print("Intercept:", round(my_model.beta[0], 4))
print()
cf_coefs = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': my_model.beta[1:]
})
print(cf_coefs.sort_values(by='Coefficient', ascending=False, key=abs).to_string(index=False))

# Predict on training set
y_train_pred_cf = my_model.predict(X_train_all)
mse_train_cf = mean_squared_error(y_train_all, y_train_pred_cf)
r2_train_cf = r2_score(y_train_all, y_train_pred_cf)

# Predict on testing set
y_test_pred_cf = my_model.predict(X_test_all)
mse_test_cf = mean_squared_error(y_test_all, y_test_pred_cf)
r2_test_cf = r2_score(y_test_all, y_test_pred_cf)


Intercept: 520.4148

      Feature  Coefficient
   sqft_above   132.944524
        grade    92.231475
          lat    78.375737
sqft_basement    75.449424
     yr_built   -67.643117
   waterfront    63.742900
         view    48.200109
sqft_living15    45.577658
  sqft_living   -38.390120
    bathrooms    18.527633
 yr_renovated    17.271380
    condition    12.964269
   sqft_lot15   -12.930091
     bedrooms   -12.521962
     sqft_lot    10.881868
       floors     8.043721
         long    -1.035203


In [15]:
print("Problem 3 (Closed-Form)")
print("Train MSE:", round(mse_train_cf, 2), "Train R2:", round(r2_train_cf, 2))
print("Test MSE:", round(mse_test_cf, 2), "Test R2:", round(r2_test_cf, 2))

print("Problem 2 (sklearn)")
print("Train MSE: 31486.17  Train R2: 0.73")
print("Test MSE: 57628.15  Test R2: 0.65")


Problem 3 (Closed-Form)
Train MSE: 31486.17 Train R2: 0.73
Test MSE: 57628.15 Test R2: 0.65
Problem 2 (sklearn)
Train MSE: 31486.17  Train R2: 0.73
Test MSE: 57628.15  Test R2: 0.65


Closed Form vs SKLEARN: 

The MSE and R^2 metrocs from my closed-form implementation is identical to SKLEARN's LinearRegression from problem 2. This is expected since sklearn's LinearRegression also uses the closed-form soultion internally. Both solve the same equation. The coefficents produced by both models differ. For example, SQFT_LIVING IS 56.75 in sklearn but -38.39 in the closed-form model, while sqft_above and sqft_basement shoft in the oppsite direction. This happens because sqft_above and sqft_basement are highly correlated. When features are correlated, there are multiple ways to split the wieght among them while still producing the same predictions. Overall both methods solve the same equation so it gives the same rpedictions and error metrics. 

# *[C]* Problem 4: Polynomial Regression (15 points)

- Consider a feature $X$, a response variable $Y$, and $N$ samples of training data. Implement a polynomial regression model that fits a polynomial of degree $p$ to the data using the least-square method. Use your own implementation from Problem 3 and adapt it for polynomial
regression. If $p=2$, the model will use two features ($X$ and $X^2$), if $p=3$ the model will use 3 features ($X,X^2,X^3$), and so on for larger values of $p$.
- Consider the house price prediction problem with feature $X=$ `sqft_living`. Train a polynomial regression model for different values of $p \le 5$ using your implementation. Include a table with the MSE and $R^2$ metrics on both the training and testing data for at least 3 different values of $p$. Discuss your observations on how the MSE and $R^2$ metrics change with the degree of the polynomial.

In [16]:
class MyLinearRegression:
    def fit(self, X, y):
        # Adds intercept column
        X_b = np.c_[np.ones((X.shape[0], 1)), X]

        # Closed-form solution
        self.beta = np.linalg.solve(X_b.T @ X_b, X_b.T @ y)

    def predict(self, X):
         # Add intercept column and compute predictions
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b @ self.beta


In [17]:
def polynomial_features(X, p):
    # Generate polynomial features up to degree p (X, X^2,etc)
    X_poly = np.column_stack([X**i for i in range(1, p+1)])
    return X_poly


In [18]:
# Extract feature (sqft_living) and target (price) for training and testing sets
X_train = train[['sqft_living']].values
y_train = train['price'].values

X_test = test[['sqft_living']].values
y_test = test['price'].values


In [19]:
# Store performance results for different polynomial degrees
results = []

for p in [1, 2, 3, 5]:
    
    # Generate polynomial features of degree p
    X_train_poly = polynomial_features(X_train, p)
    X_test_poly = polynomial_features(X_test, p)

    # Train closed-form linear regression model
    model = MyLinearRegression()
    model.fit(X_train_poly, y_train)

    # Make predictions on training and testing data
    y_train_pred = model.predict(X_train_poly)
    y_test_pred = model.predict(X_test_poly)

    # Store evaluation metrics
    results.append({
        'Degree': p,
        'Train MSE': mean_squared_error(y_train, y_train_pred),
        'Test MSE': mean_squared_error(y_test, y_test_pred),
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred)
    })

# Convert results into a DataFrame for comparison
results_df = pd.DataFrame(results)
results_df


,Degree,Train MSE,Test MSE,Train R2,Test R2
0,1,57947.526161,88575.978543,0.496709,0.468736
1,2,54822.665116,71791.679479,0.523849,0.569406
2,3,53785.194716,99833.483763,0.532860,0.401216
3,5,52626.111955,570616.914821,0.542927,-2.422464


As the polynomial degree increases, the training MSE decreases and trainoing R^2 increases. This makes sense because higher polynominals are more flexible, so they fit the training data better. Testing behaves differently. While moving from degree 1 to degree 2 improves both test MSE and test R^2, increasing the degree anymore leads to increase in test error and a decline in test R^2. For example, degree 5, the model overfits the data, resulting in a very large test MSE and a negative R^2 vaklue. This demonstrates the bias-variance tradeoff. Higher degree polynomials increase model flexibility and reduce bias but can significantly increase variance and hurt generalization performance. 

# *[C]* Problem 5:  Gradient descent (20 points)

In this problem, you will implement your own gradient descent algorithm and apply it to linear regression on the same house prediction dataset.

1. Write code for gradient descent for training linear regression using the algorithm from class.
2. Vary the value of the learning rate (at least 3 different values $\alpha \in \{0.01,0.1,0.5\}$) and report the value of the model parameter $\theta$ after different number of iterations (10, 50, and 100). Include in a table the MSE and $R^2$ metrics on the training and testing set for the different number of iterations and different learning rates. You can choose more values of the learning rates to observe how the  behavior of the algorithm changes.
3. Write some observations about the behavior of the algorithm: How do the metrics change with different learning rates; How many iterations are needed; Does the algorithm converge to the optimal solution, etc.

In [49]:
class LinearRegressionGD:
    
    def __init__(self, alpha=0.01, n_iters=1000):
        self.alpha = alpha  # learning rate
        self.n_iters = n_iters # number of iterations
        self.theta = None # parameters
        self.loss_history = []
    
    def fit(self, X, y):
        N, d = X.shape
        
        # Add intercept column
        X = np.c_[np.ones((N, 1)), X]
        
        # Initialize theta
        self.theta = np.zeros(X.shape[1])
        
        for _ in range(self.n_iters):
            
            # Predictions
            y_pred = X @ self.theta
            
            # Compute gradient
            gradient = (1/N) * (X.T @ (y_pred - y))
            
            # Update rule
            self.theta = self.theta - self.alpha * gradient
            
            # Store loss (MSE)
            loss = (1/(2*N)) * np.sum((y_pred - y)**2)
            self.loss_history.append(loss)
    
    def predict(self, X):
        N = X.shape[0]
        X = np.c_[np.ones((N, 1)), X]
        return X @ self.theta


In [51]:
# Use all features for gradient descent on the house prediction dataset
X_train_gd = train[feature_cols].values
y_train_gd = train['price'].values

X_test_gd = test[feature_cols].values
y_test_gd = test['price'].values

In [52]:
# Define learning rates and number of iterations to test
learning_rates = [0.01, 0.1, 0.5]
iterations_list = [10, 50, 100]

# Store results for comparison
results = []

# Loop over different learning rates and iteration counts
for alpha in learning_rates:
    for n_iter in iterations_list:
        
        # Initialize and train gradient descent model
        model = LinearRegressionGD(alpha=alpha, n_iters=n_iter)
        model.fit(X_train_gd, y_train_gd)

        # Generate predictions on training and testing data
        y_train_pred_gd = model.predict(X_train_gd)
        y_test_pred_gd = model.predict(X_test_gd)

        # Store model parameters and performance metrics
        results.append({
            "alpha": alpha,
            "iterations": n_iter,
            "theta_0": round(model.theta[0], 4),
            "Train MSE": round(mean_squared_error(y_train_gd, y_train_pred_gd), 2),
            "Test MSE": round(mean_squared_error(y_test_gd, y_test_pred_gd), 2),
            "Train R2": round(r2_score(y_train_gd, y_train_pred_gd), 4),
            "Test R2": round(r2_score(y_test_gd, y_test_pred_gd), 4)
        })

# Convert results into a DataFrame for easier comparison
results_df = pd.DataFrame(results)

# Display results table
results_df

,alpha,iterations,theta_0,Train MSE,Test MSE,Train R2,Test R2
0,0.01,10,49.7610,2.947987e+05,3.505251e+05,-1.560400e+00,-1.102400e+00
1,0.01,50,205.5607,1.382959e+05,1.703767e+05,-2.011000e-01,-2.190000e-02
2,0.01,100,329.9262,7.011899e+04,9.748624e+04,3.910000e-01,4.153000e-01
3,0.10,10,338.9574,6.649932e+04,9.355929e+04,4.224000e-01,4.388000e-01
4,0.10,50,517.7327,3.157898e+04,5.801232e+04,7.257000e-01,6.521000e-01
5,0.10,100,520.4010,3.149769e+04,5.772519e+04,7.264000e-01,6.538000e-01
6,0.50,10,519.9066,6.118299e+08,6.850231e+08,-5.312921e+03,-4.107652e+03
7,0.50,50,520.4148,1.649496e+25,1.842083e+25,-1.432635e+20,-1.104850e+20
8,0.50,100,-150582.1234,5.698752e+45,6.364111e+45,-4.949532e+40,-3.817086e+40


The results show that the behavior of the gradient descent is highly sensitive to the choice of the learning rate. When the learning rate is small like alpha = 0.01 then the algorthim converges slowly. The MSE decreases and the R^2 increases gradually as the number of iterations grow from 10 to 100. When learning rate is moderate like alpha = 0.1 then the algorithm converges much fasterm reaching strong performance within about 50 iterations. Additional ierations only show minor imrpoveemnt. However, when the learning rate is too large lue alpha = 0.5 then the algrothim becomes unstable and diverges. It leads to extremely alrge MSE values and higher negative R^2 scores. This shows that the parmaters updates are oevrshooting the minium rather than approaching it. Overall gradient descent converges to the otpimal soutlion when the learning rate is chosen approately. Too small elads to too slow conevrgence, whiel too large elads to divergence.

# *A/C]* Problem 6: Ridge regularization (20 points)

In this problem, you will derive the optimal parameters for ridge regression and train ridge regression models with different regularization levels. In ridge regression, the loss function includes a regularization term:

$J(\theta) = \sum_{i=1}^N(h_{\theta}(x_i)-y_i)^2 + \lambda \sum_{j=1}^d \theta_j^2$

1. **[A]** Write the derivation of the closed form solution for parameter $\theta$ that minimizes the loss function $J(\theta)$ in ridge regression. done on paper.

2. **[C]** Modify your implementation from Problem 5 to implement ridge regression with gradient descent.


In [ ]:
#  Ridge Regression with Gradient Descent
class RidgeRegressionGD:
    def __init__(self, alpha=0.01, n_iters=5000, lam=1.0):
        self.alpha = alpha
        self.n_iters = n_iters
        self.lam = lam
        self.theta = None
        self.loss_history = []

    def fit(self, X, y):
        N, d = X.shape
        Xb = np.c_[np.ones((N, 1)), X]
        self.theta = np.zeros(d + 1)

        for _ in range(self.n_iters):
            y_pred = Xb @ self.theta
            error = y_pred - y

            grad = (1/N) * (Xb.T @ error)

            # Ridge penalty gradient (no regularization on intercept)
            reg = (self.lam / N) * self.theta
            reg[0] = 0.0

            self.theta -= self.alpha * (grad + reg)

            sse = np.sum(error**2)
            penalty = self.lam * np.sum(self.theta[1:]**2)
            self.loss_history.append(sse + penalty)

    def predict(self, X):
        N = X.shape[0]
        Xb = np.c_[np.ones((N, 1)), X]
        return Xb @ self.theta

### Part 3: Simulated Data with Ridge Regression

Simulate $N=1000$ values of $X_i \sim \text{Uniform}(-2,2)$ and $Y_i = 1 + 2X_i + e_i$ where $e_i \sim N(0,2)$. Fit with linear regression and ridge regression for $\lambda \in \{1,10,100,1000,10000\}$.

In [ ]:
# Simulate data and compare models 
np.random.seed(42)
N = 1000
X_sim = np.random.uniform(-2, 2, size=(N, 1))
e = np.random.normal(0, 2, size=N)
y_sim = 1 + 2 * X_sim.flatten() + e

# Linear regression (lambda = 0)
lr = LinearRegressionGD(alpha=0.01, n_iters=5000)
lr.fit(X_sim, y_sim)
y_pred_lr = lr.predict(X_sim)

In [31]:
print("Linear Regression (lambda = 0)")
print("Intercept:", round(lr.theta[0], 4))
print("Slope:", round(lr.theta[1], 4))
print("MSE:", round(mean_squared_error(y_sim, y_pred_lr), 4))
print("R2:", round(r2_score(y_sim, y_pred_lr), 4))
print()

lambdas = [1, 10, 100, 1000, 10000]

for lam in lambdas:
    ridge = RidgeRegressionGD(alpha=0.01, n_iters=5000, lam=lam)
    ridge.fit(X_sim, y_sim)
    y_pred_ridge = ridge.predict(X_sim)

    print("Ridge Regression (lambda =", lam, ")")
    print("Intercept:", round(ridge.theta[0], 4))
    print("Slope:", round(ridge.theta[1], 4))
    print("MSE:", round(mean_squared_error(y_sim, y_pred_ridge), 4))
    print("R2:", round(r2_score(y_sim, y_pred_ridge), 4))
    print()


Linear Regression (lambda = 0)
Intercept: 1.1948
Slope: 1.9226
MSE: 3.8999
R2: 0.5639

Ridge Regression (lambda = 1 )
Intercept: 1.1947
Slope: 1.9212
MSE: 3.8999
R2: 0.5639

Ridge Regression (lambda = 10 )
Intercept: 1.1942
Slope: 1.9086
MSE: 3.9001
R2: 0.5639

Ridge Regression (lambda = 100 )
Intercept: 1.1897
Slope: 1.7913
MSE: 3.9234
R2: 0.5613

Ridge Regression (lambda = 1000 )
Intercept: 1.1631
Slope: 1.1094
MSE: 4.8021
R2: 0.463

Ridge Regression (lambda = 10000 )
Intercept: 1.1288
Slope: 0.2308
MSE: 7.8044
R2: 0.1273



Observations:

As the regularization paramter $\lambda$ increases:

- the slope shrinks toward zero: With no regularization, the model estimates slope as 1.92. At $\lambda$ = 1 and $\lambda$ = 10, it remains around 1.91-1.92 but as $\lambda$ = 1000 it drops to 1.11 and at $\lambda$ = 10000 to only 0.23. This demonstrates ridge regression's penalty on large coefficients. 

- The intercept is minimally affected. Since ridge doesn't regularize the intercept, it stays near 1.19 across most $\lambda$ values. It only drops at 1.13 at v = 10000.

- MSE increases and R^2 decreases with largr $\lambda$. For $\lambda$ less than or equal to 10, MSE, remains 3.90 and r^2 0.56. At $\lambda$ = 1000, MSE rises to 4.80 and R^2 drops to 0.46. At $\lambda$ = 10000, MSE jumps to 7.80 and R^2 falls to 0.13, indicating severe underfitting.

From this we can see that mild regularization has insignificant impact but excessive $\lambda$ introduces bias and leads to underfitting. 